In [3]:
import warnings
warnings.filterwarnings('ignore')

### Create a Virtual Environment
Create a folder in the current directory which will contain the Python executable files, and a copy of the pip library which you can use to install other packages. 

	virtualenv env

Activate Virtual Env.

	source env/bin/activate
	
Install Pandas OR maybe add REQUIREMENTS.txt
    
    pip install jupyter
    pip install pandas
	pip install requests
    pip install ipykernel

    

### Getting a Dataframe from Carto's SQL API 

In [25]:
import json
import pandas as pd 
import requests
import io

cred = json.load(open('credentials.json')) # modify credentials.json.sample
username = cred['username']
api_key  = cred['api_key']

tablename = 'yellow_tripdata_2015_04_verysmall'

url = 'https://%s.carto.com/api/v2/sql?q=SELECT * FROM %s&format=csv' % (username , tablename)

s  = requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8')))

print df.head(3)

   cartodb_id  the_geom  the_geom_webmercator  vendorid  \
0           1       NaN                   NaN         2   
1           2       NaN                   NaN         2   
2           3       NaN                   NaN         2   
3           4       NaN                   NaN         2   
4           5       NaN                   NaN         2   

     tpep_pickup_datetime   tpep_dropoff_datetime  passenger_count  \
0  2015-04-27 15:16:29+00  2015-04-27 15:26:14+00                1   
1  2015-04-27 15:16:29+00  2015-04-27 15:25:03+00                1   
2  2015-04-27 15:16:29+00  2015-04-27 15:31:48+00                1   
3  2015-04-27 15:16:29+00  2015-04-27 15:25:26+00                1   
4  2015-04-27 15:16:29+00  2015-04-27 15:45:38+00                1   

   trip_distance  pickup_longitude  pickup_latitude      ...       \
0           1.05        -73.988869        40.758396      ...        
1           2.67        -74.008568        40.704479      ...        
2           2.89 

### Get columns for desired output

In [26]:
cols = []

for i in df.columns:
	cols.append(i)

del cols[:3] #delete cartodb_id, the_geom, the_geom_webmercator,
cols = cols + ['geoid']    
c = ', '.join(cols) #list of cols to keep to string for SQL statement
    
points = 'yellow_tripdata_2015_04_verysmall'
poly   = 'tl_2014_census_tracts'    
lat    = 'yellow_tripdata_2015_04_verysmall.pickup_latitude'
lng    = 'yellow_tripdata_2015_04_verysmall.pickup_longitude'

url = "https://%s.carto.com/api/v2/sql?q=SELECT %s FROM %s, %s  WHERE ST_INTERSECTS(ST_SetSRID(ST_Point(%s, %s),4326), %s.the_geom)&format=csv" % (username, c, points, poly, lng, lat, poly )

s  = requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8')))

df = df.rename(columns=lambda x: x.replace('geoid', 'pickup_geoid'))

dfp = df[['pickup_geoid']]
dfp['count'] = 1
dfp = dfp.groupby(['pickup_geoid'],as_index=False).sum()

print dfp.head(3)

if not os.path.exists('temp'): #create temp folder
    os.makedirs('temp')
 
dfp.to_csv('temp/pickup_geoid.csv', index=False)
os.system('curl -v -F file=@temp/pickup_geoid.csv "https://'+username+'.carto.com/api/v1/imports/?api_key="'+api_key)

try:
    shutil.rmtree('temp') #delete temp folder
except:
    raise

   pickup_geoid  count
0   36061000700      1
1   36061004900      1
2   36061008200      1
3   36061010000      1
4   36061010200      1
5   36061011000      1
6   36061011202      2
7   36061011401      1
8   36061011500      1
9   36061011700      1


In [6]:
from IPython.display import display, HTML
display(HTML('<iframe width="100%" height="520" frameborder="0" src="index.html" allowfullscreen webkitallowfullscreen mozallowfullscreen oallowfullscreen msallowfullscreen></iframe>'))